In [81]:
import pandas as pd
import requests
from requests_html import HTMLSession
import io
from scrapy.selector import Selector
import re

In [7]:
df = pd.read_json('journals.json')

In [139]:
def download_file(url, file_name):
    session = HTMLSession()
    r = session.get(url)
    pattern = re.compile("pdfUrl\s\=\s\"([\w\d\W]+)\";")
    download_url = re.search(pattern, Selector(r).css('body script').get()).group(1).replace('\\', '')
    with open('pdf/' + file_name + '.pdf', 'wb') as f:
        f.write(session.get(download_url).content)
        f.close()
    return print(f'Downloaded... {file_name}')


In [175]:
# invalid_file_name_pattern = '[\\\/\?\%\>\<\:]'
# invalid_file_name_pattern_compiled = re.compile(invalid_file_name_pattern)

for i,x in df.iterrows():
    url = df.iloc[i]['article_pdf_url']
    if url:
        file_name = str(i)
        download_file(url=url, file_name=file_name)

Downloaded 109
Downloaded 111
Downloaded 112
Downloaded 113
Downloaded 114
Downloaded 115
Downloaded 116
Downloaded 117
Downloaded 118
Downloaded 119
Downloaded 120
Downloaded 121
Downloaded 122
Downloaded 123
Downloaded 124
Downloaded 125
Downloaded 126
Downloaded 127
Downloaded 128
Downloaded 129
Downloaded 130
Downloaded 131
Downloaded 132
Downloaded 133
Downloaded 134
Downloaded 135
Downloaded 136
Downloaded 137
Downloaded 138
Downloaded 139
Downloaded 140
Downloaded 141
Downloaded 142
Downloaded 143
Downloaded 144
Downloaded 145
Downloaded 146
Downloaded 147
Downloaded 148
Downloaded 149
Downloaded 150
Downloaded 151


In [183]:
df = df.reset_index().rename({'index':'key'}, axis=1)


In [256]:
def parse_pdf(file):
    doc = fitz.open('pdf/' + str(file) + '.pdf')
    print(len(doc), 'total pages has been extracted')
    return ''.join([page.get_text() for page in doc])


In [258]:
for i,x in df.iterrows():
    if df.iloc[i]['article_pdf_url']:
        df.loc[df['key'] == i, 'text'] = parse_pdf(i)

17 total pages has been extracted
17 total pages has been extracted
26 total pages has been extracted
15 total pages has been extracted
19 total pages has been extracted
20 total pages has been extracted
20 total pages has been extracted
16 total pages has been extracted
17 total pages has been extracted
14 total pages has been extracted
18 total pages has been extracted
14 total pages has been extracted
19 total pages has been extracted
13 total pages has been extracted
19 total pages has been extracted
20 total pages has been extracted
15 total pages has been extracted
16 total pages has been extracted
25 total pages has been extracted
17 total pages has been extracted
19 total pages has been extracted
19 total pages has been extracted
5 total pages has been extracted
12 total pages has been extracted
16 total pages has been extracted
16 total pages has been extracted
9 total pages has been extracted
20 total pages has been extracted
16 total pages has been extracted
4 total pages ha

In [262]:
df = df.drop(columns=['key'])

In [263]:
df.to_json('journal_with_full_text.json', 'records')